# RAG_Catalogo_Productos.ipynb


## 1) Requisitos e instalación

Este notebook usa paquetes que pueden no estar instalados en tu entorno. Ejecuta la siguiente celda para instalar dependencias necesarias. (Si estás en Colab, ejecuta la celda; en un entorno local con pip también funciona.)


In [1]:
# Ejecuta esta celda una sola vez para instalar dependencias (puede tardar).
import sys, subprocess
reqs = [
    "pandas",
    "openpyxl",
    "sentence-transformers",
    "faiss-cpu",
    "transformers",
    "accelerate",
    "torch",
    "tqdm",
    "nbformat"
]
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade"] + reqs)
print("Instalación finalizada. Reinicia el kernel si es necesario.")

Instalación finalizada. Reinicia el kernel si es necesario.


## 2) Cargar y explorar el Excel

La ruta detectada del archivo subido es `/data/Fichas_tecnicas-2025_10_30-22_24.xlsx`. La siguiente celda leerá el archivo sin asumir cabeceras y mostrará las primeras filas para ayudarte a confirmar dónde están los encabezados (si los hay).

In [8]:
# --- Step 2: Smart multi-schema loader ---

schema_map = {
    "Figuras de accion": [
        "Título", "Personaje", "Materiales", "Es articulada", "Es coleccionable",
        "Es bobblehead", "Incluye pilas", "Peso", "Unidad de Peso"
    ],
    "Calzoncillos": [
        "Título", "Talla", "Tipo de calzoncillo", "Composición", "Material principal", "Tiro del calzoncillo"
    ],
    "Medias": [
        "Título", "Talla", "Tipo de medias", "Tipo de largo", "Material principal", "Composición"
    ],
    "Panties": [
        "Título", "Talla", "Tipo de pantie", "Tiro de la pantie", "Material principal", "Composición"
    ],
    "Morrales": [
        "Título", "Capacidad de la mochila", "Altura", "Ancho", "Profundidad", "Peso",
        "Materiales", "Con compartimento para portátil", "Con ruedas", "Es a prueba de agua"
    ]
}

valid_dfs = []

for sheet in tqdm(xls.sheet_names):
    if sheet.lower() in ["hidden", "presentacion"]:
        continue
    try:
        df = pd.read_excel(xls, sheet_name=sheet, header=3)
        # Find best schema match
        keys = schema_map.get(sheet, [])
        if "Título" in df.columns:
            df = df[[c for c in keys if c in df.columns]]
            df["categoria_origen"] = sheet
            valid_dfs.append(df)
            print(f"✅ {sheet}: {df.shape}")
        else:
            print(f"⚠️ Skipping {sheet}: missing 'Título'.")
    except Exception as e:
        print(f"❌ Error reading {sheet}: {e}")

df_catalogo = pd.concat(valid_dfs, ignore_index=True)
print("\n📊 Data combinada final:", df_catalogo.shape)
display(df_catalogo.head(5))


  0%|          | 0/38 [00:00<?, ?it/s]c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


✅ Figuras de accion: (3448, 10)


  5%|▌         | 2/38 [00:00<00:17,  2.01it/s]c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


✅ Calzoncillos: (408, 7)
✅ Medias: (41, 7)
✅ Panties: (81, 7)


 13%|█▎        | 5/38 [00:01<00:06,  5.39it/s]c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


...

c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


✅ Termos: (1, 1)


100%|██████████| 38/38 [00:01<00:00, 25.70it/s]



📊 Data combinada final: (4195, 26)


,Título,Personaje,Materiales,Es articulada,Es coleccionable,Es bobblehead,Incluye pilas,Peso,Unidad de Peso,categoria_origen,...,Tipo de largo,Tipo de pantie,Tiro de la pantie,Capacidad de la mochila,Altura,Ancho,Profundidad,Con compartimento para portátil,Con ruedas,Es a prueba de agua
0,Base Figuras De Acción Neca Myth Cloth Figuart...,No aplica,PVC,Sí,Sí,No,No,226.796,g,Figuras de accion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Figura De Acción Uchiha Sasuke Anime Heroes Na...,Uchiha Sasuke,PVC,Sí,Sí,No,No,226.796,g,Figuras de accion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Figura Acción Goku Ultra Instinto Plata Dragon...,Goku ultra instinct,PVC,Sí,Sí,No,No,226.796,g,Figuras de accion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Figura De Acción Naruto Uzumaki Anime Heroes B...,Naruto Uzumaki,PVC,Sí,Sí,No,No,226.796,g,Figuras de accion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Figura De Acción Goku Ultra Instinct Dragon Ba...,Goku ultra instinct,Plástico,Sí,Sí,No,No,226.796,g,Figuras de accion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.1  Build Embeddings + Simple RAG Search

Create a texto_producto column that joins all descriptive fields per product.

Use LangChain + Chroma (free, local vector DB) to embed the products.

Test retrieval with a natural-language query like:
“¿Qué figuras tienen control remoto?” or “Muéstrame los morrales con ruedas.”

In [9]:
# --- Step 1: Combine fields into unified text for embeddings ---

df_catalogo = df_catalogo.fillna("")

def combine_fields(row):
    # Combine all column data into a text string (ignore empties)
    text_parts = [f"{col}: {str(row[col])}" for col in row.index if row[col] and col != "categoria_origen"]
    return " | ".join(text_parts)

df_catalogo["texto_producto"] = df_catalogo.apply(combine_fields, axis=1)
df_catalogo["id"] = df_catalogo.index.astype(str)

print("✅ Columnas combinadas. Ejemplo:")
display(df_catalogo[["categoria_origen", "texto_producto"]].head(3))


✅ Columnas combinadas. Ejemplo:


,categoria_origen,texto_producto
0,Figuras de accion,Título: Base Figuras De Acción Neca Myth Cloth...
1,Figuras de accion,Título: Figura De Acción Uchiha Sasuke Anime H...
2,Figuras de accion,Título: Figura Acción Goku Ultra Instinto Plat...


## 3) Local Vector
 Using lanchain to import HuggingFaceEmbeddings and Chroma

In [11]:
# --- Step 2: Build embeddings & local vectorstore ---

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create or load the Chroma vector database
vectorstore = Chroma.from_texts(
    texts=df_catalogo["texto_producto"].tolist(),
    embedding=embedding_model,
    metadatas=df_catalogo[["categoria_origen", "Título"]].to_dict(orient="records"),
    persist_directory="./chroma_db"
)

print("✅ Vector store creada con", len(df_catalogo), "productos")


C:\Users\danie\AppData\Local\Temp\ipykernel_2600\2686432378.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\danie\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Vector store creada con 4195 productos


## 4) Simple Query


In [12]:
# --- Step 3: Query the RAG database ---

query = "¿Qué figuras de acción son coleccionables de Naruto?"
results = vectorstore.similarity_search(query, k=3)

for i, r in enumerate(results, 1):
    print(f"\n🔹 Result {i}:")
    print("Categoría:", r.metadata.get("categoria_origen"))
    print("Título:", r.metadata.get("Título"))
    print("Texto:", r.page_content[:500])



🔹 Result 1:
Categoría: Figuras de accion
Título: Figura De Acción Anime Heroes Naruto Orochimaru
Texto: Título: Figura De Acción Anime Heroes Naruto Orochimaru | Materiales: Plástico | Es articulada: Sí | Es coleccionable: No | Es bobblehead: No | Incluye pilas: No | Peso: 144.0 | Unidad de Peso: g

🔹 Result 2:
Categoría: Figuras de accion
Título: Figura Naruto Shippuden Modo Sabio De Los 6 Caminos Bandai
Texto: Título: Figura Naruto Shippuden Modo Sabio De Los 6 Caminos Bandai | Personaje: Naruto | Materiales: Plástico | Es articulada: Sí | Es coleccionable: Sí | Es bobblehead: No | Incluye pilas: No | Peso: 350.0 | Unidad de Peso: g

🔹 Result 3:
Categoría: Figuras de accion
Título: Figura Naruto Shippuden Modo Sabio De Los 6 Caminos Bandai
Texto: Título: Figura Naruto Shippuden Modo Sabio De Los 6 Caminos Bandai | Personaje: Naruto | Materiales: Plástico | Es articulada: Sí | Es coleccionable: Sí | Es bobblehead: No | Incluye pilas: No | Peso: 350.0 | Unidad de Peso: g


## Set up a RetrievalQA chain

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# --- 1️⃣ Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# --- 2️⃣ Vector DB (persistente)
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# --- 3️⃣ LLM local
llm = OllamaLLM(model="mistral", temperature=0.1, num_ctx=1024)

# --- 4️⃣ Prompt
prompt = ChatPromptTemplate.from_template("""
Eres un asistente experto en el catálogo de productos de la tienda.
Responde solo con base en la información del contexto.
Si no sabes la respuesta, di: "No tengo esa información en el catálogo."

Contexto:
{context}

Pregunta:
{question}

Respuesta en español:
""")

# --- 5️⃣ RAG Chain (moderno)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

query = "¿Qué figuras de acción de Naruto tengo disponibles?"
response = rag_chain.invoke(query)

print("🧠 Pregunta:", query)
print("💬 Respuesta:", response)




🧠 Pregunta: ¿Qué figuras de acción de Naruto tengo disponibles?
💬 Respuesta:  En el catálogo, solo hay una figura de acción de Naruto disponible, que es la Figura Naruto Shippuden Modo Sabio De Los 6 Caminos Bandai.


In [ ]:
import langchain
print(langchain.__version__)


1.0.3
